In [19]:
import os
import yaml
import pandas as pd
from PIL import Image
import json

In [9]:
with open("data_config.yaml") as file:
    yaml_data = yaml.safe_load(file)

dataset = "MIMIC"
original_data_path = yaml_data[dataset]["images_path_train"]
train_data_path = yaml_data[dataset]["train_csv"]

train_df = pd.read_excel(train_data_path)
train_df['path'] = train_df['path'].apply(lambda x: os.path.join(original_data_path, x))
train_df.rename(columns={'path': 'image'}, inplace=True)

train_df.head()

,subject_id,study,text,dicom_id,findings,image,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,10000032,s53911762,"The absence of focal consolidation, effusion, ...",68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,Single frontal view of the chest provided.\n \...,/raid/s2198939/MIMIC_Dataset/physionet.org/fil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,10000764,s57375967,"The image shows a chest X-ray, which reveals b...",096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4,PA and lateral views of the chest provided. ...,/raid/s2198939/MIMIC_Dataset/physionet.org/fil...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN
2,10000898,s50771383,The image is a PA (posteroanterior) chest X-ra...,2a280266-c8bae121-54d75383-cac046f4-ca37aa16,PA and lateral views of the chest provided. ...,/raid/s2198939/MIMIC_Dataset/physionet.org/fil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,10000980,s51967283,The image shows a new triangular opacity in th...,943486a3-b3fa9ff7-50f5a769-7a62fcbb-f39b6da4,A triangular opacity in the right lung apex is...,/raid/s2198939/MIMIC_Dataset/physionet.org/fil...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,10000980,s54935705,The image shows mild pulmonary edema with supe...,6ad819bb-bae74eb9-7b663e90-b8deabd7-57f8054a,There is mild pulmonary edema with superimpose...,/raid/s2198939/MIMIC_Dataset/physionet.org/fil...,NaN,NaN,-1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN


In [12]:
def prepare_data_format_i2t(ID, PATH, DESC):
    DATA_FORMAT = {
                  "id": ID,
                  "image": [PATH],
                  "conversations": [
                    {
                      "from": "user",
                      "value": "<ImageHere> Describe the following image in detail."
                    },
                    {
                      "from": "assistant",
                      "value": DESC
                    }
                  ]
        }
    return DATA_FORMAT

def prepare_data_format_t2i(ID, PATH, DESC):
    DATA_FORMAT = {
                  "id": ID,
                  "image": [PATH],
                  "conversations": [
                    {
                      "from": "user",
                      "value": "Given the following description of a medical scan, generate an appropriate image. Description: "+DESC
                    },
                    {
                      "from": "assistant",
                      "value": "<ImageHere>"
                    }
                  ]
        }
    return DATA_FORMAT

In [21]:
data = []

for i in range(len(train_df)):
    ID = i
    PATH = train_df['image'][i]
    DESC = train_df['text'][i]

    data.append(prepare_data_format_i2t(ID, PATH, DESC))

In [22]:
with open('vl_data.json', 'w') as f:
    for item in data:
        f.write(json.dumps(item) + '\n')

In [23]:
len(train_df)

110273